In [1]:
import os
import json
from tqdm import tqdm

In [2]:
with open('../data/document/역도/chunk_with_overlap.json', 'r', encoding='utf-8') as f:
    origin_data = json.load(f)

In [3]:
section01 = origin_data[0][:5]
print(section01[0])

{'metadata': {'heading': {'heading1': 'Ⅰ. 역도의 이해', 'heading2': '1. 역도경기의 역사', 'heading3': '가. 근대올림픽 대회 이전의 역도경기', 'heading4': None, 'heading5': None, 'default': None}, 'index': 0, 'summary': "역도의 이해에 대한 문서에서는 역도경기의 역사에 대해 다루고 있다. 역도의 기원을 정확히 추측하는 것은 어렵지만, 기원전 1800년경 아일랜드에서 '무거운 물체를 들기'와 '던지기' 경기가 행해졌다고 전해진다. 기원전 6세기 후반부터 5세기까지 여러 역사적 기록이 존재하며, 이 시기를 '힘의 시대'라고 부른다. 이 시기에 크로토나의 밀로가 훈련으로 무거운 물체를 어깨에 짊어지고 걷는 방법을 사용했다고 전해진다. 고대 그리스 이전에도 돌을 들거나 던지는 소박한 형태의 역기가 존재했으며, 동서양을 불문하고 원시적인 힘의 자랑 방법으로 행해졌음을 알 수 있다. 근대적 형태의 역도는 1800년대부터 시작되었으며, 구츠무츠와 얀의 덤벨 운동이 소개된 후, 얀의 제자인 아이젤린에 의해 '덤벨 체조 소서'가 발행되면서 근대 역도가 태동했다고 할 수 있다. 덤벨의 발전은 체조를 개입시켜 진보하였고, 유럽에서는 다양한 역기자가 배출되었다. 유젠 산도우는 육체미의 포즈와 역기를 소개하여 청소년들과 근력에 관심 있는 사람들의 지지를 얻었고, 아더 섹손은 한 손으로 168.5kg 바벨을 머리 위로 들어올려 세상 사람들의 이목을 집중시켰다. 그의 명성은 현재까지 이어지고 있다.", 'filename': '역도의 이해_edited.json', 'page': [1, 2, 3], 'chunk_id': 1}, 'page_content': "역도의 기원을 정확하게 추측하는 것은 쉬\n운 일이 아니나 기원전 1800년경에 아일랜드\n에서는‘무거운 물체를 들어올리기’,‘무거운 물\n체를 던지기’경기가 행해졌다고 전해지고 있\n다. 아마 이것이 역도의 기원이라

# RAGAS Customization의 필요성
## 1 실제 사용자 질의와 ragas 평가용 질의의 간극 해소
* ragas의 기본 데이터 생성 방식은 대부분의 문서에 적용 가능한 범용적인 문서-질의-응답 패턴을 기반으로 설계됨
* summary embedding의 유사도를 중심으로 문서를 선택하고 질의를 생성하는 구조적 특성으로 인해 다음과 같은 문제가 발생
  * Multi-Hop 평가 데이터 생성에서 페르소나 생성이나 문서 선택 단계에서 summary embedding을 주로 활용
  * 이로 인해, 동일 키워드에 대한 서로 다른 주제나 테마를 가진 문서들 기반 평가 데이터셋 생성 불가능
  * 결과적으로 실제 사용자 질의의 복합적 의도나 문맥적 요규사항을 충분히 반영하지 못하는 한계 발생
  * 질의 예시
    * ex) "**sumo 데드리프트의 마무리 동작에서 허리**에 과도한 무게 집중이 나타나고 있어. 이 현상이 나타나는 **과학적 이유**와 이를 해결하기 위한 **연습 방법**을 작성해줘."
      * 중심 키워드: sumo 데드리프트, 마무리 동작, 허리
      * 주제: 과학적 이유, 연습 방법
    * 이와 같이, 실제 사용자 질의는 단일 키워드에 대한 복합 주제 흐름을 포함하는 경우가 많음.
    * 그러나, RAGAS 기본 데이터 생성 방식은 이를 추분히 반영하지 못함으로, 커스터마이징이 필수

## 2. 변별력 강화를 위한 합성 데이터 설계
* 좋은 평가 데이터는 다양한 RAG 시스템이나 LLM 모델 간의 성능 차이를 명확하게 드러낼 수 있어야 함
* 이는 '변별력'을 지닌 질의로 정의
* RAGAS의 Multi-Hop 관련 기본 합성 데이터 생성은 2 ~ 3개의 Chunk를 사용
* Multi-Hop에 사용되는 chunk의 갯수 조정, 시나리오 생성 신규 방안 도입을 통해 변별력을 가진 합성 데이터 생성  

## RAGAS Customization 확인 절차
1. ragas 기반의 기본 합성 데이터셋 생성
2. 기본 합성 데이터셋 기반 검색 절차의 hyper parameter tuning 진행
3. custom 합성 데이터셋 생성
4. custom 합성 데이터셋 기반 검색 절차의 hyper parameter tuning 진행
5. 두 실험의 평가지표 비교
   * 성능 분산(분포)의 폭 비교
   * 정렬 결과 차이 분석(Rank Sensitivity)
   * 통계적 유의성 테스트

## hyper parameter 관련 주요 설정(auto_rag)
1. 검색 평가 지표: [retrieval_f1, retrieval_ndcg, retrieval_map]
2. bm25 tokenizer: ko_kiwi
3. rrf_k(num_chunk): [3, 5, 10]
4. 생성 평가 지표: bert_score 및 g_eval
   * 여기서 ragas의 주요 지표를 사용해도 좋을거 같음

## 1) 정보 추출
https://docs.ragas.io/en/latest/howtos/customizations/testgenerator/_testgen-custom-single-hop/

In [9]:
from ragas.testset.graph import KnowledgeGraph
from ragas.testset.graph import Node, NodeType


kg = KnowledgeGraph()
for doc in section01:
    kg.nodes.append(
        Node(
            type=NodeType.DOCUMENT,
            properties={
                "page_content": doc['page_content'],
                "document_metadata": doc['metadata'],
            },
        )
    )

In [16]:
from langchain_openai import ChatOpenAI
from ragas.llms import LangchainLLMWrapper
from ragas.testset.transforms import apply_transforms

from ragas.testset.transforms.extractors.llm_based import (
    NERExtractor,
    SummaryExtractor
)

llm = LangchainLLMWrapper(ChatOpenAI())

ner_extractor = NERExtractor(llm=llm)
summary_extractor = SummaryExtractor(llm=llm)

transforms = [
    ner_extractor,
    summary_extractor
]

apply_transforms(kg, transforms=transforms)

In [27]:
kg.nodes[0].properties.keys()

dict_keys(['page_content', 'document_metadata', 'entities', 'summary'])

In [36]:
kg.relationships[0].properties

{'entities_overlap_score': 0.03125,
 'overlapped_items': [('올림픽', '올림픽'), ('체육사', '체육사')]}

In [35]:
kg.relationships[0]

Relationship(Node(id: a1bf21) -> Node(id: ca77c5), type: entities_overlap, properties: ['entities_overlap_score', 'overlapped_items'])

In [ ]:
from ragas.testset.persona import Persona

personas = [
    Persona(
        name="aspiring coach",
        role_description="A university student majoring in physical education who wants to become a weightlifting coach",
    ), # 체육교육을 전공하며 역도 지도자를 꿈꾸는 대학생

    Persona(
        name="high school athlete",
        role_description="A high school student training for national weightlifting competitions",
    ), # 전국 대회를 준비 중인 고등학생 역도 선수

    Persona(
        name="beginner lifter",
        role_description="Someone new to weightlifting looking for basic guidance on form and training routines",
    ), # 역도를 처음 시작하며 기본 자세와 훈련법을 찾는 초보자
]


# RAGAS 합성 데이터셋 생성 관련 시나리오 출력
* 개선점
  * 한번 생성된 지식그래프를 저장하는 코드 필요
  * 각종 단계를 출력하는 코드
  * 페르소나에서 role_description도 출력하도록 변경 필요
  * 시나리오 자체도 출력하도록 수정 필요
  * 나중에는 지식그래프 입력하고 transform만 입력하면, 문제만 생성 하도록 (?)
  * summary를 실행하지 않도록 하는 방법(?)
  * 영어로 출력되는 문제 해결하기
* 지금 해야하는 것
  * 작성된 코드를 분석하기
  * 분석 결과를 기준으로 개선하는 것

In [20]:
import sys
sys.path.append('../code/ragas_custom/')

from custom_testset_utilities import *

from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings

# RAGAS 실험
### 평가 항목
1. Generation 
   1. Faithfulness
      * 주어진 문맥에 대한 생성된 답변의 사실적 일관성 측정
      * 답변과 검색된 문맥(retrieved context)를 기준으로 계산
      * (0, 1) 범위 스케일이며, 값이 높을수록 좋음
      * 생성된 답변이 신뢰할 수 있다고(faithful) 간구되려면 답변에서 제시된 모든 주장이 주어진 문맥(given context)에서 추론될 수 있어야 함
      * 생성된 답변에서 주장의 집합(claims)를 식별 -> 각 주장마다 주어진 맥락 기반 여부 확인
      * 점수: context 기반의 답변 내 주장 수 / 전체 주장 수
      * 예시
        * 아이슈타인의 출생일자와 출생지는 어디인가?
          * 답변 1: 아이슈타인은 독엘에서 1879/3/14에 태어났습니다.
            * 
          * 답변 2: 아이슈타인은 독엘에서 1879/4/14에 태어났습니다.
            * 
          * context: 
* 
   1. 
1. Retriever
   1. 
## 1. 